In [5]:
pip install gdcm
pip install pylibjpeg 
pip install pylibjpeg-libjpeg

SyntaxError: invalid syntax (2528679311.py, line 1)

In [2]:
import os
import pandas as pd 
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns
import pylibjpeg as jpeg

In [3]:
STUDY_PATH=os.path.join("/NAS","dumbo","protocoles","meningiome_AZ","test_perf","DICOM")

In [ ]:
import pydicom

# Path to the DICOM file
dicom_path = os.path.join(STUDY_PATH, "25033115", "27210000", "19641418")

# Read the DICOM file
dicom_data = pydicom.dcmread(dicom_path)

# Display the image
plt.imshow(dicom_data.pixel_array, cmap='gray')
plt.title("DICOM Image")
plt.axis('off')
plt.show()

In [5]:
# Créer un GIF animé avec une meilleure qualité
images[0].save(
    output_gif_path,
    save_all=True,
    append_images=images[1:],
    duration=500,
    loop=0,
    optimize=False  # Désactiver l'optimisation pour une meilleure qualité
)

print(f"GIF animé créé avec une meilleure qualité : {output_gif_path}")


GIF animé créé avec une meilleure qualité : /NAS/dumbo/protocoles/meningiome_AZ/test_perf/DICOM/25033115/screenshots/animated.gif
